In [1]:
'''
pre-processing for binary classification task.

Basing logic off of this: https://aclanthology.org/2024.findings-acl.24.pdf

Broadly - if any one of three annotators label as misogynistic, it is misogynistic.
All rewritten sentences counterfactual and non-misogynistic.
'''

'\npre-processing for binary classification task.\n\nBasing logic off of this: https://aclanthology.org/2024.findings-acl.24.pdf\n\nBroadly - if any one of three annotators label as misogynistic, it is misogynistic.\nAll rewritten sentences counterfactual and non-misogynistic.\n'

In [31]:
import pandas as pd

In [4]:
biasly_raw = pd.read_csv('biasly_dataset.csv')

In [10]:
biasly_raw

,Unnamed: 0,datapoint_id,datapoint,is_misogynistic,why_unclear,misogynistic_inferences,other_inferences,inferences_explanation,original_severity,rewrite_possible,rewrite,rewrite_severity,annotator_id,annotator_background
0,0,9dd77afa-dfc8-4adb-87b2-b51f72ccc5fc,ititmeansthatit's whatdoesthatmean? ititmeanst...,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A1,Linguistics
1,1,9dd77afa-dfc8-4adb-87b2-b51f72ccc5fc,ititmeansthatit's whatdoesthatmean? ititmeanst...,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A2,Gender Studies
2,2,9dd77afa-dfc8-4adb-87b2-b51f72ccc5fc,ititmeansthatit's whatdoesthatmean? ititmeanst...,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A3,Gender Studies
3,3,e5cdb361-842d-434c-a056-491119d380d9,She doesn't want me to call her. Look what you...,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A1,Linguistics
4,4,e5cdb361-842d-434c-a056-491119d380d9,She doesn't want me to call her. Look what you...,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A2,Gender Studies
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29995,668c6b0f-d4a6-4a39-8dca-ab0ff418e8b9,"Angels can't ride as good. Oh, sweet little gi...",Yes,NaN,Trivialization (infantilizing or paternalistic...,NaN,NaN,179.0,Possible,"Angels can't ride as good. Oh, girl. Yeah.",0.0,A8,Linguistics
29996,29996,668c6b0f-d4a6-4a39-8dca-ab0ff418e8b9,"Angels can't ride as good. Oh, sweet little gi...",Yes,NaN,Trivialization (infantilizing or paternalistic...,NaN,NaN,230.0,Not Possible,NaN,NaN,A10,Linguistics
29997,29997,89669210-b819-435d-919f-a088727533f0,"When I buy a 12-pack, I just have to trust' em...",No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A4,Gender Studies
29998,29998,89669210-b819-435d-919f-a088727533f0,"When I buy a 12-pack, I just have to trust' em...",No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A8,Linguistics


In [24]:
#gather all datapoints that have "is_misogynistic" == "Yes"
misogyny_datapoints = []

for index in biasly_raw.index:
    
    if biasly_raw.loc[index]["is_misogynistic"]=="Yes":
        misogyny_datapoints.append(
            biasly_raw.loc[index]['datapoint']
        )

misogyny_datapoints = list(set(misogyny_datapoints))

In [35]:
#all datapoints for non-misogynistic data

non_misogyny_datapoints = []
for index in biasly_raw.index:
    
    if biasly_raw.loc[index]["is_misogynistic"]=="No":
        non_misogyny_datapoints.append(
            biasly_raw.loc[index]['datapoint']
        )
        
    if not pd.isna(biasly_raw.loc[index]["rewrite"]):
        non_misogyny_datapoints.append(
            biasly_raw.loc[index]['rewrite']
        )

non_misogyny_datapoints = list(set(non_misogyny_datapoints))

In [41]:
print(f"Number of misogynistic datapoints: {len(misogyny_datapoints)}, which is {len(misogyny_datapoints)/(len(misogyny_datapoints)+len(non_misogyny_datapoints))*100}% of dataset")
print(f"Number of non-misogynistic datapoints: {len(non_misogyny_datapoints)}, which is {len(non_misogyny_datapoints)/(len(misogyny_datapoints)+len(non_misogyny_datapoints))*100}% of dataset")

Number of misogynistic datapoints: 3159, which is 20.97888165759065% of dataset
Number of non-misogynistic datapoints: 11899, which is 79.02111834240935% of dataset


In [51]:
# turn dataset into labelled csv
biasly_prepared_dataset = []

for item in misogyny_datapoints:
    biasly_prepared_dataset.append(
        {
            "datapoint": item,
            "misogynistic_label": 1
        }
    )

for item in non_misogyny_datapoints:
    biasly_prepared_dataset.append(
        {
            "datapoint": item,
            "misogynistic_label": 0
        }
    )
    
biasly_prepared_df = pd.DataFrame(biasly_prepared_dataset, index=None)
biasly_prepared_df.to_csv("biasly_prepared_df.csv", index=False)